# Søk om utsatt frist for et set av klienter

In [1]:
from docs.test.testinnsending.skatteetaten_api import get_access_token
from requests import post
from faker import Faker
import jwt
import uuid

token = get_access_token()
jwt_token = token["Authorization"].replace("Bearer ", "")


#henter innlogget bruker fra tokenet
identifikator = jwt.decode(jwt_token, options={"verify_signature": False})["sub"]
inntektsår = "2024"

/Users/jon.solvberg/workspace/skattemeldingen/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


The token is good, expires at 15:34:53 (3599 seconds from now)


127.0.0.1 - - [09/Dec/2024 14:34:53] "GET /token?code=Miros249epN0NHta5aOedw.6e4RYUwgIMPqYoeh7p88Dw&iss=https%3A%2F%2Ftest.idporten.no&state=FfHjWOzhWlRKj_5vu_K8lw HTTP/1.1" 200 -


In [2]:
def lag_indentifikator(identifikator: str) -> dict():
    fake = Faker()

    if len(identifikator) > 9:
        identifikator_dict = {"foedselsnummer" : identifikator}
    else:
        identifikator_dict = {"organisasjonsnummer" : identifikator}

    kandidat = {
        "identifikator": identifikator_dict,
        "naering":  True, #kan ikke mikse True / False av næring i en og samme request
        "navn": fake.name(),
        "epost": fake.email(),
    }
    return kandidat


parter = ["21909899726", "18853449683"]

klienter = [lag_indentifikator(kandidat) for kandidat in parter]

payload = {"referanse": str(uuid.uuid4()),
           "klienter": klienter}

url_path = f"/api/skattemelding/v2/utsattfristsoeknad/bulk/{identifikator}"
base_url = "https://idporten-api-sbstest.sits.no"

In [4]:
utsatt_frist_request = post(base_url+url_path, headers=token, json=payload)
print(f"http: {utsatt_frist_request.status_code}")
print(utsatt_frist_request.text)

http: 200
{"gjeldendeInntektsaar":2024,"innsender":"13840649827","klient":"21909899726","beskrivelse":"Ingen endring ble gjort for innsendt skattepliktig."}


In [3]:
url_path = f"/api/skattemelding/v2/utsattfristsoeknad/{identifikator}"

payload = {"referanse": str(uuid.uuid4()),
           "klient": lag_indentifikator("21909899726")}

utsatt_frist_request_singel = post(base_url+url_path, headers=token, json=payload)
print(f"http: {utsatt_frist_request_singel.status_code}")
print(utsatt_frist_request_singel.text)

http: 200
{"gjeldendeInntektsaar":2024,"innsender":"13840649827","klient":"21909899726","beskrivelse":"Ingen endring ble gjort for innsendt skattepliktig."}
